In [23]:
import pandas as pd

In [24]:
athletes=pd.read_csv("athlete_events.csv")

In [25]:
athletes.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [26]:
athletes = athletes[athletes["Season"] == "Summer"]

In [27]:
athletes

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
26,8,"Cornelia ""Cor"" Aalten (-Strannood)",F,18.0,168.0,NaN,Netherlands,NED,1932 Summer,1932,Summer,Los Angeles,Athletics,Athletics Women's 100 metres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271106,135565,Fernando scar Zylberberg,M,27.0,168.0,76.0,Argentina,ARG,2004 Summer,2004,Summer,Athina,Hockey,Hockey Men's Hockey,NaN
271107,135566,"James Francis ""Jim"" Zylker",M,21.0,175.0,75.0,United States,USA,1972 Summer,1972,Summer,Munich,Football,Football Men's Football,NaN
271108,135567,Aleksandr Viktorovich Zyuzin,M,24.0,183.0,72.0,Russia,RUS,2000 Summer,2000,Summer,Sydney,Rowing,Rowing Men's Lightweight Coxless Fours,NaN
271109,135567,Aleksandr Viktorovich Zyuzin,M,28.0,183.0,72.0,Russia,RUS,2004 Summer,2004,Summer,Athina,Rowing,Rowing Men's Lightweight Coxless Fours,NaN


In [28]:
def team_summary(data):
    return pd.Series({
        'team': data.iloc[0,:]["NOC"],
        'country': data.iloc[-1,:]["Team"],
        'year': data.iloc[0,:]["Year"],
        'events': len(data['Event'].unique()),
        'athletes': data.shape[0],
        'age': data["Age"].mean(),
        'height': data['Height'].mean(),
        'weight': data['Weight'].mean(),
        'medals': sum(~pd.isnull(data["Medal"]))
    })

team = athletes.groupby(["NOC", "Year"]).apply(team_summary)

In [29]:
team = team.reset_index(drop=True)
team = team.dropna()

In [34]:
team[team.isnull().any(axis=1)]

,team,country,year,events,athletes,age,height,weight,medals


In [33]:
team

,team,country,year,events,athletes,age,height,weight,medals
3,AFG,Afghanistan,1960,13,16,23.312500,170.687500,69.437500,0
4,AFG,Afghanistan,1964,8,8,22.000000,161.000000,64.250000,0
5,AFG,Afghanistan,1968,5,5,23.200000,170.200000,70.000000,0
6,AFG,Afghanistan,1972,8,8,29.000000,168.333333,63.750000,0
7,AFG,Afghanistan,1980,11,11,23.636364,168.363636,63.181818,0
...,...,...,...,...,...,...,...,...,...
2805,ZIM,Zimbabwe,2000,19,26,24.961538,178.960000,71.080000,0
2806,ZIM,Zimbabwe,2004,11,14,25.071429,177.785714,70.500000,3
2807,ZIM,Zimbabwe,2008,15,16,26.062500,171.928571,63.714286,4
2808,ZIM,Zimbabwe,2012,8,9,27.333333,174.444444,65.222222,0


In [37]:
team["medals"] = team["medals"].fillna(0) 

In [39]:
def prev_medals(data):
    data = data.sort_values("year", ascending=True)
    data["prev_medals"] = data["medals"].shift(1)
    #data["prev_3_medals"] = data.rolling(3, closed="left", min_periods=1).mean()["medals"]
    return data

team = team.groupby(["team"]).apply(prev_medals)
team = team.reset_index(drop=True)
team = team[team["year"] > 1960]
team = team.round(1)

In [40]:
team[team["team"] == "USA"]

,team,country,year,events,athletes,age,height,weight,medals,prev_medals
2497,USA,United States,1964,159,492,23.5,177.2,71.8,169,125.0
2498,USA,United States,1968,167,529,23.5,177.2,72.1,166,169.0
2499,USA,United States,1972,185,578,23.6,177.6,71.7,171,166.0
2500,USA,United States,1976,189,554,23.0,177.3,71.0,164,171.0
2501,USA,United States,1984,217,693,24.6,177.2,71.6,352,164.0
2502,USA,United States,1988,230,715,25.2,177.1,71.4,207,352.0
2503,USA,United States,1992,248,734,25.7,177.0,71.9,224,207.0
2504,USA,United States,1996,263,839,26.4,177.5,73.2,259,224.0
2505,USA,United States,2000,265,764,26.6,177.3,73.6,242,259.0
2506,USA,United States,2004,254,726,26.5,177.0,73.6,263,242.0


In [41]:
team

,team,country,year,events,athletes,age,height,weight,medals,prev_medals
1,AFG,Afghanistan,1964,8,8,22.0,161.0,64.2,0,0.0
2,AFG,Afghanistan,1968,5,5,23.2,170.2,70.0,0,0.0
3,AFG,Afghanistan,1972,8,8,29.0,168.3,63.8,0,0.0
4,AFG,Afghanistan,1980,11,11,23.6,168.4,63.2,0,0.0
5,AFG,Afghanistan,2004,5,5,18.6,170.8,64.8,0,0.0
...,...,...,...,...,...,...,...,...,...,...
2608,ZIM,Zimbabwe,2000,19,26,25.0,179.0,71.1,0,0.0
2609,ZIM,Zimbabwe,2004,11,14,25.1,177.8,70.5,3,0.0
2610,ZIM,Zimbabwe,2008,15,16,26.1,171.9,63.7,4,3.0
2611,ZIM,Zimbabwe,2012,8,9,27.3,174.4,65.2,0,4.0


In [42]:
team.to_csv("teams.csv",index=False)